In [1]:
from dash.dependencies import Input, Output
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt

import dash
import dash_core_components as dcc
import dash_html_components as html

import pandas as pd
import os
import numpy as np
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import shapefile
import descartes
from shapely.geometry import Point
import gmaps
import gmaps.datasets
import geopandas
import plotly.graph_objects as go # or plotly.express as px
import plotly.express as px
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.figure_factory as ff
import re
from datetime import datetime as dt
import json
import dash_bootstrap_components as dbc

from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
     counties = json.load(response)

In [3]:
df = pd.read_csv('/Users/jiaqilu/Desktop/ENV19/data_visualization/GHC_sample_waterquality.csv')

/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
XCH4 = df.loc[df['var'] =='XCH4']          
    

In [12]:
XCH4 = XCH4[['date','value','county','fips','unit','var']]
XCH4['date'] = pd.to_datetime(XCH4['date'])
# XCH4.to_csv('XCH4_FINAL.csv')

In [104]:
fig = go.Figure()
temp = XCH4.loc[XCH4['date'].dt.year == 2020,:]
fig = px.line(temp, x="date", y="value")

fig.update_layout(xaxis_title='Time',
           yaxis_title='CH4 Concentration (nmol/mol)')

fig.update_traces(marker_size=20)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Closure timeline
fig.update_layout(
                shapes=[
                    # 1st highlight during April 4 - May 31
                    dict(
                        type="rect",
                        # x-reference is assigned to the x-values
                        xref="x",
                        # y-reference is assigned to the plot paper [0,1]
                        yref="paper",
                        x0="2020-03-31",
                        y0=0,
                        x1="2020-05-31",
                        y1=1,
                        fillcolor="LightSalmon",
                        opacity=0.5,
                        layer="below",
                        line_width=0,
                    )
                ]
            )

y_max = max(temp['value'])

fig.add_shape(
        # Vertical line on 2020-03-31
            type="line",
            x0= '2020-03-31',
            y0= min(temp['value']),
            x1= '2020-03-31',
            y1= y_max,
            line=dict(
                color="LightSeaGreen",
                width=4,
                dash="dashdot",
            ),
          
    )


fig.add_shape(
        # Vertical line on 2020-05-05
            type="line",
            x0= '2020-05-05',
            y0= min(temp['value']),
            x1= '2020-05-05',
            y1= y_max,
            line=dict(
                color="LightBlue",
                width=4,
                dash="dashdot",
            ),
          
    )

fig.add_shape(
        # Vertical line on 2020-05-18
            type="line",
            x0= '2020-05-18',
            y0= min(temp['value']),
            x1= '2020-05-18',
            y1= y_max,
            line=dict(
                color="Orange",
                width=4,
                dash="dashdot",
            ),
          
    )


fig.add_shape(
        # Vertical line on 2020-06-03
            type="line",
            x0= '2020-06-03',
            y0= min(temp['value']),
            x1= '2020-06-03',
            y1= y_max,
            line=dict(
                color="Red",
                width=4,
                dash="dashdot",
            ),
    )


#Add text to lines 
fig.add_trace(go.Scatter(
    x=['2020-03-31', '2020-04-28','2020-05-18','2020-06-10'],
    y=[y_max, y_max,y_max,y_max],
    text=["<i>Stay at Home Order</i>",
          "<i>Phase 1 Reopening</i>",
          "<i>Phase 2 Reopening</i>",
          "<i>Phase 3 Reopening</i>"],
    mode="text",
    textfont = dict(size=8),
    showlegend =False
))



#step function

# Only shows the step function of the first two Quaters data in 2020 
step = XCH4.set_index('date')['value']
step = step.resample('Q').mean()
step = pd.DataFrame(step)
step = step.loc[(step.index >= '2019-12-31' )&(step.index <= '2020-07')  ,:]
fig.add_trace(go.Scatter(x=step.index, y=step['value'], name='Step function',
                    line_shape='hv', line= dict(color ='firebrick')))




fig.show()

,value
date,
2019-12-31,1805.459883
2020-03-31,1786.805562
2020-06-30,1795.399508
